# Monte Carlo Project

## Iacob Jessica, Mourre Grégoire

In this Jupter file, we will implement the different methods presented and described in https://github.com/gregoiremrr/Monte-Carlo-for-American-Options.


In [11]:
import numpy as np
from numpy.random import default_rng
from tqdm import tqdm

from sklearn.neural_network import MLPRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

# Methods for the lower bound

In [3]:
n = 10000
m = 12
seed = 0
rng = default_rng(seed=seed)

r, sigma, x, K, T = 0.06, .4, 80, 100, 0.5
dt = T/m

def g(x,t=0):
    return np.exp(-r*t*dt) * np.maximum(K-x,0)

def phi(x):
    # Canonical basis
    #return np.array([1, x, x**2, x**3], dtype=object)
    #return np.array([1, x, x**2, x**3, g(x)], dtype=object)
    #return np.array([1, x, x**2, x**3, x**4, g(x)], dtype=object)
    #return np.array([1, x, x**2, x**3, g(x), g(x)**2], dtype=object)
    return np.array([1, x, x**2, x**3, g(x), g(x)**2, g(x)**3], dtype=object)
    # Legendre's basis
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2], dtype=object)
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2, g(x)], dtype=object)
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2, (35*x**4-30*x**2+3)/8, g(x)], dtype=object)
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2, g(x), g(x)**2], dtype=object)
    #return np.array([1, x, (3*x**2-1)/2, (5*x**3-3*x)/2, g(x), g(x)**2, g(x)**3], dtype=object)
    # Hermite's basis
    #return np.array([1, x, x**2-1, x**3-3*x], dtype=object)
    #return np.array([1, x, x**2-1, x**3-3*x, g(x)], dtype=object)
    #return np.array([1, x, x**2-1, x**3-3*x, x**4-6*x**2+3, g(x)], dtype=object)
    #return np.array([1, x, x**2-1, x**3-3*x, g(x), g(x)**2], dtype=object)
    #return np.array([1, x, x**2-1, x**3-3*x, g(x), g(x)**2, g(x)**3], dtype=object)
    # Laguerre's basis
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6], dtype=object)
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6, g(x)], dtype=object)
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6, (x**4-16*x**3+72*x**2-96*x+24)/24, g(x)], dtype=object)
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6, g(x), g(x)**2], dtype=object)
    #return np.array([1, 1-x, (x**2-4*x+2)/2, (-x**3+9*x**2-18*x+6)/6, g(x), g(x)**2, g(x)**3], dtype=object)
l = len(phi(0))

def reg(x, *alphas):
    alpha = np.array(alphas)
    return np.sum(alpha * phi(x))

neural_network_ci = [Pipeline([('preprocessing',StandardScaler()), ('nn',MLPRegressor(hidden_layer_sizes=(3, 5, 5, 3), max_iter=5000, random_state=0))]) for _ in range(m)]
#neural_network_ci = [Pipeline([('preprocessing',StandardScaler()), ('svr',SVR())]) for _ in range(m)]

# Longstaff and Schwartz's algorithm
Approximation of the Value Functions (Steps 2 to 4 in Longstaff & Schwartz's algorithm)

In [4]:
rng = default_rng(seed=seed)

# Step 1
B = rng.normal(size=m*n).reshape(m,n)
X = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B]), axis=0)

# Step 2
V2 = np.zeros(n*(m+1)).reshape((m+1),n)
V2[-1,:] = g(X[-1,:],m)

# Step 3
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    neural_network_ci[i].fit(X[i,:].reshape(-1,1), V2[i+1,:].ravel())
    _1 = g(X[i,:],i)
    _2 =  neural_network_ci[i].predict(X[i,:].reshape(-1, 1))
    V2[i,:] = _1 * (_1 >= _2) + V2[(i+1),:] * (_1 < _2)

# Step 4 (not used)
_1 = g(x)
_2 = np.mean(V2[1,:])
V02 = np.mean(_1 * (_1 >= _2) + V2[1,:] * (_1 < _2))

Longstaff & Schwartz's algorithm (Step 5)

In [5]:
rng = default_rng(seed=seed)

# Step 5
B2 = rng.normal(size=m*n).reshape(m,n)
X2 = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B2]), axis=0)

V3 = np.zeros(n*(m+1)).reshape((m+1),n)
V3[-1,:] = g(X2[-1,:],m)

for i in range(m-1, 0, -1):
    _1 = g(X2[i,:],i)
    _2 =  neural_network_ci[i].predict(X2[i,:].reshape(-1, 1))
    V3[i,:] = _1 * (_1 >= _2) + V3[(i+1),:] * (_1 < _2)

_1 = g(x)
_2 = np.mean(V3[1,:])
V03_ = _1 * (_1 >= _2) + V3[1,:] * (_1 < _2)

V03 = np.mean(V03_)
std = np.std(V03_)
conv_interval = V03 + np.array([-1,1]) * 1.96 * std / np.sqrt(n)
final_lowerbound = conv_interval[0]

print("Estimator:", V03)
print("Standard deviation:", std / np.sqrt(n))
print("Condidence interval 95%:", conv_interval)
print("Error:", 100 * 1.96 * std / (V03 * np.sqrt(n)), "%")

Estimator: 21.47304533846319
Standard deviation: 0.12675819004468372
Condidence interval 95%: [21.22459929 21.72149139]
Error: 1.1570135887644952 %


# Martingales from Approximate Value Functions

In [6]:
# One trajectory (just to get the idea of the next reel method)

rng = default_rng(seed=seed)

n2 = 1000

B_upperbound = rng.normal(size=m)
X_upperbound = np.cumprod(np.concatenate([[x], 1 + r * dt + sigma * np.sqrt(dt) * B_upperbound]), axis=0)
Normal = rng.normal(size=m*n2).reshape(m,n2)
M = np.zeros(m+1)
Mi = 0

for i in range(1, m):
    V_upperbound = max(g(X_upperbound[i],i), neural_network_ci[i].predict(X_upperbound[i].reshape(-1, 1)))
    X_next = X_upperbound[i-1] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:])
    V_Ynext = np.maximum(g(X_next,i), neural_network_ci[i].predict(X_next.reshape(-1, 1)))
    delta = V_upperbound - np.mean(V_Ynext)
    Mi += delta
    M[i] = Mi

X_next = X_upperbound[-2] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:])
delta = g(X_upperbound[-1],m) - np.mean(g(X_next,m))
Mi += delta
M[-1] = Mi

V0_upperbound = np.max(g(X_upperbound,np.arange(0,m+1)) - M)
print(V0_upperbound)

21.591534798235138


In [7]:
# Monte-Carlo method for n trajectories

rng = default_rng(seed=seed)

n2 = 1000

Normal = rng.normal(size=m*n*n2).reshape(m,n2,n)
M = np.zeros(n*(m+1)).reshape(m+1,n)
Mi = np.zeros(n)

for i in tqdm(range(1, m)):
    V_upperbound = np.maximum(g(X2[i,:],i), neural_network_ci[i].predict(X2[i,:].reshape(-1, 1)))
    X_next = X2[i-1,:] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:,:])
    V_Ynext = np.maximum(g(X_next,i), neural_network_ci[i].predict(X_next.reshape(-1, 1)).reshape(n2,n))
    Mi += V_upperbound - np.mean(V_Ynext, axis=0)
    M[i,:] = Mi

X_next = X2[-2,:] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:,:])
Mi += g(X2[-1,:],m) - np.mean(g(X_next,m), axis=0)
M[-1,:] = Mi

V0_upperbound_ = np.max(g(X2.T,np.arange(0,m+1)).T - M, axis=0)
V0_upperbound = np.mean(V0_upperbound_)
std_upperbound = np.std(V0_upperbound_)
conv_interval_upperbound = V0_upperbound + np.array([-1,1]) * 1.96 * std_upperbound / np.sqrt(n)
final_upperbound = conv_interval_upperbound[1]

print("Estimator:", V0_upperbound)
print("Standard deviation:", std_upperbound / np.sqrt(n))
print("Condidence interval 95%:", conv_interval_upperbound)
print("Error:", 100 * 1.96 * std_upperbound / (V0_upperbound * np.sqrt(n)), "%")

100%|██████████| 11/11 [00:19<00:00,  1.74s/it]

Estimator: 21.71627734807821
Standard deviation: 0.00891986545497708
Condidence interval 95%: [21.69879441 21.73376028]
Error: 0.08050613837505734 %


In [8]:
print("Confidence interval (lower & upper bounds):", [final_lowerbound, final_upperbound])

Confidence interval (lower & upper bounds): [21.22459928597561, 21.733760284369968]


# Martingales from Stopping Rules

In [9]:
rng = default_rng(seed=seed)

# Step 1

n_subpaths = 500
Normal = rng.normal(size=m*m*n*n_subpaths).reshape(n,m,m,n_subpaths)

subpaths = np.zeros(m*(m+1)*n*n_subpaths).reshape(n,m,m+1,n_subpaths)
for k in range(n):
    for i in range(m):
        subpaths[k,i,i+1:,:] = X2[i,k] * np.cumprod(1 + r * dt + sigma * np.sqrt(dt) * Normal[k,i,i:,:], axis=0)

subpaths_values = np.zeros(m*(m+1)*n*n_subpaths).reshape(n,m,m+1,n_subpaths)
for i in tqdm(range(m)):
    subpaths_values[:,i,m,:] = g(subpaths[:,i,m,:],m)
    for j in range(m-1,i,-1):
        _1 = g(subpaths[:,i,j,:],j)
        _2 = neural_network_ci[j].predict(subpaths[:,i,j,:].reshape(-1, 1)).reshape(n,n_subpaths)
        subpaths_values[:,i,j,:] =  _1 * (_1 >= _2) + subpaths_values[:,i,j+1,:] * (_1 < _2)

# approximation of E[ h_{\tau_{i+1}}(X_{\tau_{i+1}}) | X_i ] = V_iplus1[:,i]
V_iplus1 = np.zeros(n*m).reshape(n,m)
for i in range(m):
    V_iplus1[:,i] = np.mean(subpaths_values[:,i,i+1,:], axis = 1)

# approximation of E[ h_{\tau_i}(X_{\tau_i}) | X_i ] = V_i[:,i]
V_i = np.zeros(n*(m+1)).reshape(n,m+1)
for i in range(1,m):
    _1 = g(X2[i,:],i)
    _2 = neural_network_ci[i].predict(X2[i,:].reshape(-1, 1))
    V_i[:,i] = _1 * (_1 >= _2) + V_iplus1[:,i] * (_1 < _2)

V_i[:,m] = g(X2[m,:],m)

Mk = np.zeros(n)
M2 = np.zeros(n*(m+1)).reshape(n,m+1)
for i in range(1,m+1):
    Mk += V_i[:,i] - V_iplus1[:,i-1]
    M2[:,i] = Mk

V0_upperbound2_ = np.max(g(X2.T,np.arange(0,m+1)).T - M2.T, axis=0)
V0_upperbound2 = np.mean(V0_upperbound2_)
std_upperbound2 = np.std(V0_upperbound2_)
conv_interval_upperbound2 = V0_upperbound2 + np.array([-1,1]) * 1.96 * std_upperbound2 / np.sqrt(n)
final_upperbound2 = conv_interval_upperbound2[1]

print("Estimator:", V0_upperbound2)
print("Standard deviation:", std_upperbound2 / np.sqrt(n))
print("Condidence interval 95%:", conv_interval_upperbound2)
print("Error:", 100 * 1.96 * std_upperbound2 / (V0_upperbound2 * np.sqrt(n)), "%")

100%|██████████| 12/12 [01:02<00:00,  5.17s/it]


Estimator: 21.724349084476234
Standard deviation: 0.006401605748968468
Condidence interval 95%: [21.71180194 21.73689623]
Error: 0.05775614827025647 %


In [10]:
print("Confidence interval (lower & upper bounds):", [1, final_upperbound])#final_lowerbound
print("Standard deviation:", std_upperbound / np.sqrt(n))
print("Confidence interval (lower & upper bounds):", [1, final_upperbound2])#final_lowerbound
print("Standard deviation 2:", std_upperbound2 / np.sqrt(n))

Confidence interval (lower & upper bounds): [1, 21.733760284369968]
Standard deviation: 0.00891986545497708
Confidence interval (lower & upper bounds): [1, 21.73689623174421]
Standard deviation 2: 0.006401605748968468
